In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import seaborn as sns
import cv2
import PIL
import pathlib
from sklearn.metrics import classification_report
from tqdm import tqdm

In [ ]:
data_dir = pathlib.Path(r"F:\Github\SMART_PARKING\smart_parking\model\model_images")
valid_dir = pathlib.Path(r"F:\Github\SMART_PARKING\smart_parking\model\valid_images")

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    directory=data_dir,
    target_size=(180, 180),
    batch_size=32,
    class_mode='binary',
    shuffle=True
)
valid_generator = valid_datagen.flow_from_directory(
    directory=valid_dir,
    target_size=(180, 180),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpus))

In [ ]:
model = keras.Sequential()
# Augmentation
model.add(keras.layers.RandomRotation(0.2))
model.add(keras.layers.RandomContrast(0.5))
model.add(keras.layers.RandomZoom(0.3))
# CNN 1
model.add(keras.layers.Conv2D(filters=16,kernel_size=(3,3),activation='relu',padding='same',input_shape=(180,180,3)))
model.add(keras.layers.MaxPool2D())
# CNN 2
model.add(keras.layers.Conv2D(filters=32,kernel_size=(3,3),padding='same',activation='relu'))
model.add(keras.layers.MaxPool2D())
# CNN 3
model.add(keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same'))
model.add(keras.layers.MaxPool2D())
# Dense
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512,activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(256,activation='relu'))
model.add(keras.layers.Dense(1,activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_generator,validation_data=valid_datagen ,epochs=10)

In [ ]:
# # Augmentation block (cleanly separated)
# data_augmentation = keras.Sequential([
#     layers.RandomRotation(0.1),
#     layers.RandomContrast(0.1),
#     layers.RandomZoom(0.1),
# ], name="augmentation")

# # Build the model
# model = models.Sequential([
#     data_augmentation,
    
#     # Conv Block 1
#     layers.Conv2D(16, (3, 3), padding='same', input_shape=(img_height, img_width, 3)),
#     layers.BatchNormalization(),
#     layers.Activation('relu'),
#     layers.MaxPooling2D(),

#     # Conv Block 2
#     layers.Conv2D(32, (3, 3), padding='same'),
#     layers.BatchNormalization(),
#     layers.Activation('relu'),
#     layers.MaxPooling2D(),

#     # Conv Block 3
#     layers.Conv2D(64, (3, 3), padding='same'),
#     layers.BatchNormalization(),
#     layers.Activation('relu'),
#     layers.MaxPooling2D(),

#     # Dense Layers
#     layers.Flatten(),
#     layers.Dense(256, activation='relu'),
#     layers.Dropout(0.5),
#     layers.Dense(128, activation='relu'),
#     layers.Dense(1, activation='sigmoid')
# ])

# # Compile the model
# model.compile(
#     optimizer='adam',
#     loss='binary_crossentropy',
#     metrics=['accuracy']
# )

# # Callbacks
# early_stop = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
# checkpoint = keras.callbacks.ModelCheckpoint("best_model.h5", save_best_only=True)

# # Train the model
# model.fit(
#     train_generator,
#     validation_data=valid_generator,
#     epochs=20,
#     callbacks=[early_stop, checkpoint]
# )

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the model
class ParkingModel(nn.Module):
    def __init__(self):
        super(ParkingModel, self).__init__()

        # Augmentation is done via transforms in PyTorch, not in the model
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

        self.fc1 = nn.Linear(64 * 22 * 22, 256)  # 180x180 image downsampled by maxpool
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)

        self.dropout = nn.Dropout(0.5)

        # For Batch Normalization
        self.bn1 = nn.BatchNorm2d(16)
        self.bn2 = nn.BatchNorm2d(32)
        self.bn3 = nn.BatchNorm2d(64)

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.max_pool2d(x, 2)  # MaxPool layer
        x = torch.relu(self.bn2(self.conv2(x)))
        x = torch.max_pool2d(x, 2)  # MaxPool layer
        x = torch.relu(self.bn3(self.conv3(x)))
        x = torch.max_pool2d(x, 2)  # MaxPool layer

        x = x.view(-1, 64 * 22 * 22)  # Flatten the tensor

        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))  # Output layer

        return x

# Instantiate the model
model = ParkingModel()

# Define the loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Define data augmentation and preprocessing
train_transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomResizedCrop(180, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

valid_transform = transforms.Compose([
    transforms.Resize((180, 180)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load the datasets
train_dataset = datasets.ImageFolder(root=r"F:\Github\SMART_PARKING\smart_parking\model\model_images", transform=train_transform)
valid_dataset = datasets.ImageFolder(root=r"F:\Github\SMART_PARKING\smart_parking\model\valid_images", transform=valid_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)



#     # Save best model
#     if valid_loss < best_valid_loss:
#         best_valid_loss = valid_loss
#         torch.save(model.state_dict(), "best_model.pth")
#         print("Saved best model")

# # Final save
# torch.save(model.state_dict(), "final_parking_model.pth")


In [ ]:
# Training loop
num_epochs = 20
best_valid_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Accuracy
        predicted = outputs.round()
        correct_predictions += (predicted.squeeze() == labels).sum().item()
        total += labels.size(0)

    # Calculate training accuracy
    train_accuracy = 100 * correct_predictions / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.2f}%")

    # Validation loop
    model.eval()
    valid_loss = 0.0
    correct_predictions = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in valid_loader:
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())

            valid_loss += loss.item()

            # Accuracy
            predicted = outputs.round()
            correct_predictions += (predicted.squeeze() == labels).sum().item()
            total += labels.size(0)

    valid_accuracy = 100 * correct_predictions / total
    print(f"Validation Loss: {valid_loss/len(valid_loader):.4f}, Validation Accuracy: {valid_accuracy:.2f}%")